# Self Organization Map for Banks

## 1) Importing the packages and data pre-processing

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('Credit_Card_Applications.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  690 non-null    int64  
 1   A1          690 non-null    int64  
 2   A2          690 non-null    float64
 3   A3          690 non-null    float64
 4   A4          690 non-null    int64  
 5   A5          690 non-null    int64  
 6   A6          690 non-null    int64  
 7   A7          690 non-null    float64
 8   A8          690 non-null    int64  
 9   A9          690 non-null    int64  
 10  A10         690 non-null    int64  
 11  A11         690 non-null    int64  
 12  A12         690 non-null    int64  
 13  A13         690 non-null    int64  
 14  A14         690 non-null    int64  
 15  Class       690 non-null    int64  
dtypes: float64(3), int64(13)
memory usage: 86.4 KB


In [3]:
df.head()

,CustomerID,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,Class
0,15776156,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,15739548,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,15662854,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,15687688,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,15715750,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [6]:
df['Class'].unique()

array([0, 1], dtype=int64)

Note that we have no null values, which is great. Additionally, The "Class" only has 2 possible options, the positive one (1) and the negative one(0). 

## 2) Adjust format and keep important information

Remember that this is an unsupervised ML model. Therefore, we are only keeping the "class" to compare later when we have our clusters. 

In [ ]:
X = df.iloc[:,1:-1].values # Since X = df.iloc[rows, columns] we want ALL the rows(:), but all the columns except the last one (:-1)
Y = pd.concat([df.iloc[:, 0], df.iloc[:, -1]], axis=1).values #we want all the rows, but only the first and the last column

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1)) # We have to normalize before everything, since we are going to use euclydian distance.
sc.fit_transform(X)
